In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os
import getpass
import dotenv


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")


pdf_folder_path = "ApicultureData"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
print(len(documents))



1243


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()




{'input': 'Quantas colméias de mel natural tem no Rio Grande do Sul?',
 'context': [Document(page_content='Tabela 20 – Distribuição da Produção e do Número de  Colméias de Mel Natural do \nTerritório Central do Estado do Rio Grande do Sul –  2002 E 2006 \n (em Kg /Quantidade/Taxa de Crescimento) \nAno \n2002 2006 2002-2006 \nMunicípios \nProdução Nº de \nColméias Produção Nº de \nColméias Taxa de Crescimento -\nColméias \n \nAgudo 12.480 832 13.460 897 7,85 \nCacequi 17.200 1.147 12.500 833 -27,33 \nCachoeira do Sul 65.500 4.367 78.500 5.233 19,85 \nCapão do Cipó 8.450 563 12.980 865 53,61 \nCerro Branco 4.600 307 6.000 400 30,43 \nDilermando de Aguiar 13.640 909 11.177 745 -18,06 \nDona Francisca 2.690 179 3.189 213 18,55 \nFaxinal do Soturno 5.010 334 5.420 361 8,18 \nFormigueiro 5.880 392 6.075 405 3,32 \nItaara 7.580 505 10.110 674 33,38 \nIvorá 10.170 678 8.500 567 -16,42 \nJaguari 59.271 3.951 59.918 3.995 1,09 \nJarí 17.250 1.150 26.000 1.733 50,72 \nJúlio de Castilhos 13.595 90

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Quantas colméias de mel natural tem no Rio Grande do Sul?"})

results

In [10]:
print(results["answer"])

Sim, as abelhas gostam do cajueiro. As abelhas solitárias do gênero Centris, conhecidas como abelhas coletoras de óleos, são os principais polinizadores do caju, e espécies de abelhas sem ferrão, como a abelha jandaíra (Melipona subnitida), também coletam néctar nas flores do cajueiro.


In [4]:
print(results["context"][0].metadata)

{'page': 38, 'source': 'ApicultureData/ESTUDO DA CADEIA DO MEL E DERIVADOS.pdf'}


In [6]:
results = rag_chain.invoke({"input": "As abelhas gostam do cajueiro?"})

results

{'input': 'As abelhas gostam do cajueiro?',
 'context': [Document(page_content='O cajueiro é uma árvore comum em \npequenos pomares, nas cidades e também muito cultivada em quase \ntodo o país. O seu fruto\xa0verdadeiro \né a castanha, um fruto seco muito apreciado no Brasil e no exterior. O “caju” é um pseudofruto, \ncarnoso,  suculento e muito rico \nem fonte de vitamina C, utilizado principalmente na produção de sucos e doces.Suas inflorescências são formadas \npor flores vermelhas, pequenas e \nperfumadas. O néctar é o recurso mais atrativo para os polinizadores, embora o pólen também seja \ncoletado por algumas espécies \nde abelhas. Abelhas solitárias do gênero Centris, também conhecidas como abelhas coletoras de óleos, são os principais polinizadores \ndo caju. Espécies de abelhas sem \nferrão como a abelha jandaíra (Melipona subnitida) também coletam néctar nas flores do \ncajueiro.\nAs abelhas do gênero Centris necessitam de óleo para construírem seus ninhos e alimentarem suas

In [7]:
print(results["context"][0].metadata)

{'page': 7, 'source': 'ApicultureData/GUIA DE PLANTAS VISITADAS POR ABELHAS NA CAATINGA.pdf'}


In [8]:
results2 = rag_chain.invoke({"input": "Quais as plantas que as abelhas preferem?"})
results2

{'input': 'Quais as plantas que as abelhas preferem?',
 'context': [Document(page_content='Biodiversidade de flores e abelhas \nTrigona spinipes e\nWaltheria rotundifolia\nMelipona subnitida e\nTarenaya\xa0spinosa\nPlebeia sp.  e\nCroton sonderianus\nMacho de abelha solitária \ndormindo na flor Turnera subulata\nPlebeia sp.  e\nCereus jamacaru\nExomalopsis sp.  e\nSenna obtusifolia\nApis mellifera\ne Senegalia polyphylla\nFrieseomelitta sp.\ne Libidibia ferrea\nXylocopa sp. e\nLibidibia ferrea\nCeblurgus longipalpis e\nVarronia leucocephala\nÁRVORES 09 ÁRVORES 08', metadata={'page': 4, 'source': 'ApicultureData/GUIA DE PLANTAS VISITADAS POR ABELHAS NA CAATINGA.pdf'}),
  Document(page_content='Muitas plantas desenvolveram compostos \nsecundários comumente associados com a defesa contra herbívoros, produzem néctar que é tóxico ou repelente para alguns visitantes florais. Outros meios de intoxicação são via pólen, secreção dos nectários extraflorais, seiva ou “honeydew” (BARKER, 1990; ADL

In [9]:
print(results["context"][0].metadata)

{'page': 7, 'source': 'ApicultureData/GUIA DE PLANTAS VISITADAS POR ABELHAS NA CAATINGA.pdf'}


In [11]:
results3 = rag_chain.invoke({"input": "Como posso iniciar minha criação de abelhas sem ferrão?"})
print(results3["answer"])
print(results3["context"][0].metadata)

Para iniciar a criação de abelhas sem ferrão, você deve primeiro escolher uma espécie adequada ao seu ambiente e aprender sobre suas necessidades específicas. Em seguida, obtenha um enxame de uma fonte confiável e instale-o em uma colmeia apropriada, preferencialmente perto de uma colônia mãe, para facilitar o estabelecimento inicial. É importante garantir que há uma boa oferta de alimento na área e monitorar o desenvolvimento da colônia regularmente para assegurar seu sucesso.
{'page': 37, 'source': 'ApicultureData/ManualdeDoencasdasAbelhaswebcomprimido.pdf'}
